In [30]:
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Conv2D,Flatten,Dropout,MaxPooling2D,LeakyReLU
from keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
early_stop = EarlyStopping(
    monitor='val_loss',      
    patience=5,              
    restore_best_weights=True
)

In [23]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="training",
    seed=123,
    image_size=(256, 256),
    batch_size=64
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/aiderdata/AIDER",
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(256, 256),
    batch_size=64
)

Found 6433 files belonging to 5 classes.
Using 5469 files for training.
Found 6433 files belonging to 5 classes.
Using 964 files for validation.


In [24]:
from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),         
    layers.RandomRotation(0.25),             
    layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.1)
])

In [25]:
normalization_layer = layers.Rescaling(1./255)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.map(lambda x, y: (data_augmentation(normalization_layer(x)), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [26]:
model= Sequential()

model.add(Conv2D(32,kernel_size=(3,3),input_shape=(256,256,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(64,kernel_size=(3,3)))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

model.add(Dense(64))
model.add(LeakyReLU(alpha=0.1))

model.add(Dense(5,activation='softmax'))

In [27]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 254, 254, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 125, 125, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 246016)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │    15,745,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,764,805 (60.14 MB)

 Trainable params: 15,764,805 (60.14 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history=model.fit(train_ds,epochs=25,validation_data=val_ds,callbacks=[early_stop])

Epoch 1/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 47s 493ms/step - accuracy: 0.5970 - loss: 2.3928 - val_accuracy: 0.7106 - val_loss: 0.7998
Epoch 2/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 43s 484ms/step - accuracy: 0.7038 - loss: 0.8129 - val_accuracy: 0.7490 - val_loss: 0.6863
Epoch 3/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 43s 487ms/step - accuracy: 0.7465 - loss: 0.6980 - val_accuracy: 0.7127 - val_loss: 0.7753
Epoch 4/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 43s 481ms/step - accuracy: 0.7527 - loss: 0.6731 - val_accuracy: 0.7459 - val_loss: 0.7371
Epoch 5/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 43s 482ms/step - accuracy: 0.7458 - loss: 0.6745 - val_accuracy: 0.7521 - val_loss: 0.6914
Epoch 6/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 43s 479ms/step - accuracy: 0.7584 - loss: 0.6342 - val_accuracy: 0.7749 - val_loss: 0.6310
Epoch 7/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 42s 476ms/step - accuracy: 0.7816 - loss: 0.5806 - val_accuracy: 0.7853 - val_loss: 0.5867
Epoch 8/25
86/86 ━━━━━━━━━━━━━━━━━━━━ 44s 491ms/step - accuracy: 0.7836 - loss: 0.5640 - val_accu